In [1]:
import ncd
from ncd import shape

## Dot-Product Attention
Using *Neural Circuit Diagrams* we can represent dot-product attention by;

<img src="Graphics/attention.png" width="700">

Each vertical section of a diagram corresponds to a shape. Columns either represent the data type we are working with, or an operation between data types. Solid lines represent axes and dashed lines separating terms represent Cartesian products. Placing a solid line adjacent to an operation, without separation, lifts it. Wirings represent Einstein operations, which includes linear contractions and rearrangements.

In [2]:
def display_columns(target: ncd.Shape, name = None):
    columns = '\n'.join([
        f'Domain:   {target.dom}',
        *(f'Morphism: {x}\n' + 
          f'Object:   {x.cod}' 
        for x in ncd.Composed.get_content(target))
    ])
    if name:
        columns = name + '\n' + columns
    return columns

In [3]:
from ncd import Duplicate
from ncd.nn import Einops, Linear, Addition, SoftMax
from itertools import starmap

L = Linear

x = shape('x')
m = shape('m^')
# Copying is implicit
linears = x >> (m @ (L('q') + L('k') + L('v')) @ 'k^')
# The axis names for einops are simply used as tags, which are attached
# to configurations.
ein1 = Einops('y k, x k -> y x')
softmax = x >> SoftMax()
ein2 = Einops('y x, x k -> y k')
linOut = L('o') @ m

# Note, the printout will contain tagged axes. These are configured
# upon composition.
section_names = ['Linears', 'Einops', 'SoftMax', 'Einops', 'Linear']
sections = [linears, ein1, softmax, ein2, linOut]
print("Printout of Individual Sections;")
print('\n'.join(starmap(display_columns, zip(sections, section_names))))

Printout of Individual Sections;
Linears
Domain:   [x m→]
Morphism: [x m→Δ3]
Object:   ([x m→], [x m→], [x m→])
Morphism: ([x→Lq], [x→Lk], [x→Lv])
Object:   ([x k→], [x k→], [x k→])
Einops
Domain:   ([y=y.CC k=k.DC→], [x=x.F7 k=k.DC→])
Morphism: y k, x k -> y x
Object:   [y=y.CC x=x.F7→]
SoftMax
Domain:   [x *=*.B1→]
Morphism: [x→◁]
Object:   [x *=*.B1→]
Einops
Domain:   ([y=y.45 x=x.DA→], [x=x.DA k=k.F1→])
Morphism: y x, x k -> y k
Object:   [y=y.45 k=k.F1→]
Linear
Domain:   *=*.FB
Morphism: Lo
Object:   [m→]


In [4]:
attention = linears @ (ein1 @ softmax + '*') @ ein2 @ linOut
print("\nPrintout of Composed Expression;")
print(display_columns(attention))


Printout of Composed Expression;
Domain:   [x m→]
Morphism: [x m→Δ3]
Object:   ([x m→], [x m→], [x m→])
Morphism: ([x→Lq], [x→Lk], [x→Lv])
Object:   ([x k→], [x k→], [x k→])
Morphism: (y k, x k -> y x, [x k→])
Object:   ([x x→], [x k→])
Morphism: ([x→◁], [x k→])
Object:   ([x x→], [x k→])
Morphism: y x, x k -> y k
Object:   [x k→]
Morphism: Lo
Object:   [m→]


In [5]:
# We can use the marches package to disassemble an algebraic expression into
#   a graph, and to then compile it into code. Currently, PyTorch is supported.
import ncd.marches

# We redefine the expression using configurable axes so that the '__init__'
# function knows which configuration parameters are required.
x_conf = ncd.Conf('x')
m_conf = ncd.Conf('m')
k_conf = ncd.Conf('k')

# We use a functor which remaps objects to make them configurable.
make_configurable = ncd.DictFunctor({
    shape('x'): x_conf,
    shape('m'): m_conf,
    shape('k'): k_conf})

# See if our functor worked ie succesfully mapped set objects to
# configurable objects.
print(display_columns(make_configurable(attention)))

Domain:   [x=x.F9 m=m→]
Morphism: [x=x.F9 m=m→Δ3]
Object:   ([x=x.F9 m=m→], [x=x.F9 m=m→], [x=x.F9 m=m→])
Morphism: ([x=x.F9→Lq], [x=x.F9→Lk], [x=x.F9→Lv])
Object:   ([x=x.F9 k=k.23→], [x=x.F9 k=k.23→], [x=x.F9 k=k.23→])
Morphism: (y k, x k -> y x, [x=x.F9 k=k.23→])
Object:   ([x=x.F9 x=x.F9→], [x=x.F9 k=k.23→])
Morphism: ([x=x.F9→◁], [x=x.F9 k=k.23→])
Object:   ([x=x.F9 x=x.F9→], [x=x.F9 k=k.23→])
Morphism: y x, x k -> y k
Object:   [x=x.F9 k=k.23→]
Morphism: Lo
Object:   [m=m→]


In [6]:
# It did! So we can compile it, with a correct __init__ function.
# 'Multilinear' is found in ncd.torch_utilities
print(ncd.marches.to_torch(make_configurable(attention), "Attention"))

class Attention(nn.Module):
    def __init__(self, k, m, x):
        self.Lq = Multilinear((m),(k))
        self.Lk = Multilinear((m),(k))
        self.Lv = Multilinear((m),(k))
        self.Lo = Multilinear((x, k),(m))
    def forward(self, a):
        a, b, c = a, a, a
        a = Lq(a)
        b = Lk(b)
        c = Lv(c)
        a = einops.einsum(a, b, "y k, x k -> y x")
        a = torch.softmax(a, dim=-1)
        a = einops.einsum(a, c, "y x, x k -> y k")
        a = Lo(a)
        return a


## Multi-Head Dot Product Attention
We represent the more intricate multi-head dot-product attention by;

<img src="Graphics/multihead.png" width="700">

This diagram has an additional $h$ axis. The linear layers output data of size ``k h``, there is additional wiring for the Einops, and the SoftMax is lifted below. We can implement these changes using our algebraic tools.

In [7]:
from ncd import Duplicate, shape
from ncd.nn import Einops, Linear, Addition, SoftMax

# Multi-Headed Attention defined symbolically.
# We piece together individual sections.
# Upon composition, axes sizes are aligned!
L = Linear
m = shape('*m^')
x = shape('x')

linears = x >> (m @ (L('q') + L('k') + L('v')) @ '*k *h')
einops = (Einops('q k h, x k h -> q x h') + '*')
softs = ((x >> SoftMax() << '*') + '*') @ Einops('q x h, x k h -> q k h')
linout = (shape('*k *h') @ L('o') @ m)

# Note, the printout will contain tagged axes. These are configured
# upon composition.
section_names = ['Linears', 'Einops', 'SoftMax + Einops', 'Linear']
sections = [linears, einops, softs, linout]
print("Printout of Individual Sections;")
print('\n'.join(starmap(display_columns, zip(sections, section_names))))

Printout of Individual Sections;
Linears
Domain:   [x m=m.8E→]
Morphism: [x m=m.8E→Δ3]
Object:   ([x m=m.8E→], [x m=m.8E→], [x m=m.8E→])
Morphism: ([x→Lq], [x→Lk], [x→Lv])
Object:   ([x k=k.25 h=h.3B→], [x k=k.25 h=h.3B→], [x k=k.25 h=h.3B→])
Einops
Domain:   ([q=q k=k.7C h=h.ED→], [x=x.E7 k=k.7C h=h.ED→], =.D4)
Morphism: (q k h, x k h -> q x h, =.D4)
Object:   ([q=q x=x.E7 h=h.ED→], =.D4)
SoftMax + Einops
Domain:   ([x *=x.F8 =h.16→], [x=x.F8 k=k.E5 h=h.16→])
Morphism: ([x→◁←=h.16], [x=x.F8 k=k.E5 h=h.16→])
Object:   ([x *=x.F8 =h.16→], [x=x.F8 k=k.E5 h=h.16→])
Morphism: q x h, x k h -> q k h
Object:   [x k=k.E5 h=h.16→]
Linear
Domain:   [k=k.98 h=h.D1→]
Morphism: Lo
Object:   [m=m.8E→]


In [8]:
multihead = linears @ einops @ softs @ (shape('*k *h') @ L('o') @ m)

print(display_columns(multihead))

Domain:   [x m=m.8E→]
Morphism: [x m=m.8E→Δ3]
Object:   ([x m=m.8E→], [x m=m.8E→], [x m=m.8E→])
Morphism: ([x→Lq], [x→Lk], [x→Lv])
Object:   ([x k=k.E5 h=h.ED→], [x k=k.E5 h=h.ED→], [x k=k.E5 h=h.ED→])
Morphism: (q k h, x k h -> q x h, [x k=k.E5 h=h.ED→])
Object:   ([x x h=h.ED→], [x k=k.E5 h=h.ED→])
Morphism: ([x→◁←=h.ED], [x k=k.E5 h=h.ED→])
Object:   ([x x =h.ED→], [x k=k.E5 h=h.ED→])
Morphism: q x h, x k h -> q k h
Object:   [x k=k.E5 h=h.ED→]
Morphism: [x→Lo]
Object:   [x m=m.8E→]


In [9]:
# We can use the "GetConfig" functor to accumulate the
# unassigned variables in its internal state. This allows
# us to quickly generate configuration parameters from an
# expression.
config = ncd.GetConfig()
config(multihead)
print(config.configs)

{k.E5, m.8E, h.ED}


In [10]:
# Marches is a package for compiling code.
# Here, it generate code for multi-headed attention.
import ncd.marches

print(ncd.marches.to_torch(multihead, "MultiHeadAttention"))

class MultiHeadAttention(nn.Module):
    def __init__(self, k, m, h):
        self.Lq = Multilinear((m),(k, h))
        self.Lk = Multilinear((m),(k, h))
        self.Lv = Multilinear((m),(k, h))
        self.Lo = Multilinear((k, h),(m))
    def forward(self, a):
        a, b, c = a, a, a
        a = Lq(a)
        b = Lk(b)
        c = Lv(c)
        a = einops.einsum(a, b, "q k h, x k h -> q x h")
        a = torch.softmax(a, dim=-2)
        a = einops.einsum(a, c, "q x h, x k h -> q k h")
        a = Lo(a)
        return a
